### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner,OpenAIChatCompletionsModel, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from openai import AsyncAzureOpenAI
import os

load_dotenv(override=True)

True

In [ ]:

client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
)

model = OpenAIChatCompletionsModel(
    model="gpt-4.1",
    openai_client=client,
)

In [18]:
from accounts import Account

In [19]:
account = Account.get("Ed")
account

Account(name='ed', balance=9609.220000000001, strategy='', holdings={'AMZN': 9}, transactions=[3 shares of AMZN at 22.044 each., 3 shares of AMZN at 70.14 each., 3 shares of AMZN at 38.076 each.], portfolio_value_time_series=[('2026-02-08 16:19:49', 10074.868), ('2026-02-08 16:19:49', 10122.868), ('2026-02-08 16:21:27', 9819.448), ('2026-02-08 16:21:27', 10215.448), ('2026-02-08 16:22:57', 9636.220000000001), ('2026-02-08 16:22:57', 10356.220000000001), ('2026-02-08 16:23:05', 10374.220000000001), ('2026-02-08 16:23:05', 9708.220000000001)])

In [20]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9392.788, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 22.044, "timestamp": "2026-02-08 16:19:49", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 70.14, "timestamp": "2026-02-08 16:21:27", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 38.076, "timestamp": "2026-02-08 16:22:57", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 72.144, "timestamp": "2026-02-08 16:24:23", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-02-08 16:19:49", 10074.868], ["2026-02-08 16:19:49", 10122.868], ["2026-02-08 16:21:27", 9819.448], ["2026-02-08 16:21:27", 10215.448], ["2026-02-08 16:22:57", 9636.220000000001], ["2026-02-08 16:22:57", 10356.220000000001], ["2026-0

In [21]:
account.report()

'{"name": "ed", "balance": 9392.788, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 22.044, "timestamp": "2026-02-08 16:19:49", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 70.14, "timestamp": "2026-02-08 16:21:27", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 38.076, "timestamp": "2026-02-08 16:22:57", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 72.144, "timestamp": "2026-02-08 16:24:23", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-02-08 16:19:49", 10074.868], ["2026-02-08 16:19:49", 10122.868], ["2026-02-08 16:21:27", 9819.448], ["2026-02-08 16:21:27", 10215.448], ["2026-02-08 16:22:57", 9636.220000000001], ["2026-02-08 16:22:57", 10356.220000000001], ["2026-02-08 16:23:05", 10374.220000

In [22]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 22.044,
  'timestamp': '2026-02-08 16:19:49',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 70.14,
  'timestamp': '2026-02-08 16:21:27',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 38.076,
  'timestamp': '2026-02-08 16:22:57',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 72.144,
  'timestamp': '2026-02-08 16:24:23',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [23]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [24]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='buy_shares', 

In [ ]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"


In [26]:
async with MCPServerStdio(
    params=params, client_session_timeout_seconds=30
) as mcp_server:
    agent = Agent(
        name="account_manager",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server],
    )
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Here are the details for your account, Ed:

- Cash balance: $9,392.79
- Holdings: 12 shares of Amazon (AMZN)

Let me know if you’d like to take any action or need more information about your portfolio.

### Now let's build our own MCP Client

In [27]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='buy_shares', ti

In [28]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(
        name="account_manager",
        instructions=instructions,
        model=model,
        tools=openai_tools,
    )
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your current cash balance is $9,392.79. If you need information about your other holdings or want to make a transaction, just let me know!

In [29]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9392.788, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 22.044, "timestamp": "2026-02-08 16:19:49", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 70.14, "timestamp": "2026-02-08 16:21:27", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 38.076, "timestamp": "2026-02-08 16:22:57", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 72.144, "timestamp": "2026-02-08 16:24:23", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-02-08 16:19:49", 10074.868], ["2026-02-08 16:19:49", 10122.868], ["2026-02-08 16:21:27", 9819.448], ["2026-02-08 16:21:27", 10215.448], ["2026-02-08 16:22:57", 9636.220000000001], ["2026-02-08 16:22:57", 10356.220000000001], ["2026-02-08 16:23:05", 10374.2200000

In [30]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9392.788, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 22.044, "timestamp": "2026-02-08 16:19:49", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 70.14, "timestamp": "2026-02-08 16:21:27", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 38.076, "timestamp": "2026-02-08 16:22:57", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 72.144, "timestamp": "2026-02-08 16:24:23", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-02-08 16:19:49", 10074.868], ["2026-02-08 16:19:49", 10122.868], ["2026-02-08 16:21:27", 9819.448], ["2026-02-08 16:21:27", 10215.448], ["2026-02-08 16:22:57", 9636.220000000001], ["2026-02-08 16:22:57", 10356.220000000001], ["2026-02-08 16:23:05", 10374.220000

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>